# COVID-19 status in Chile
> COVID-19 overview in Chile

- toc: true
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/Chile-total-confirmed-cases.png

In [1]:
#hide
import numpy as np
import pandas as pd
import altair as alt

In [2]:
#hide
from IPython.display import display_html, HTML

In [3]:
#hide
update_date = pd.to_datetime('today') - pd.offsets.Hour(48)
today = update_date.strftime('%Y-%m-%d')
today

'2020-04-27'

In [4]:
#hide
date_one_week_ago = (update_date - pd.offsets.Day(7)).strftime('%Y-%m-%d')
date_one_week_ago

'2020-04-20'

In [5]:
#hide_input
print(f"Updated on {update_date.strftime('%B %d, %Y')}.")

Updated on April 27, 2020.


In [6]:
#hide
casos_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [7]:
#hide
s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" \
+ today + "-CasosConfirmados-totalRegional.csv"
deaths_raw = pd.read_csv(s, index_col='Region')

In [8]:
#hide
s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" \
+ date_one_week_ago + "-CasosConfirmados-totalRegional.csv"
deaths_one_week_ago = pd.read_csv(s, index_col='Region')

In [9]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')

In [10]:
#hide
data_table = pd.DataFrame()
data_table["Region"] = casos_raw[today].drop("Total").reset_index()["Region"]
data_table["Confirmed cases"] = casos_raw[today].drop("Total").reset_index()[today]
data_table["Confirmed cases per 100,000 people"] = np.round(100000*(casos_raw[today]/tests_raw['Poblacion']).drop('Total'), decimals=1).values
data_table["Confirmed deaths"] = deaths_raw["Fallecidos"].drop("Total").values
data_table["Confirmed deaths per 100.000 people"] = \
np.round(100000*deaths_raw['Fallecidos'].drop('Total')/(tests_raw['Poblacion'].values), decimals=1).values

In [11]:
#hide
total_confirmed = casos_raw.iloc[-1,-1]
total_deaths = deaths_raw.iloc[-1,-1]

In [12]:
#hide
total_confirmed_one_week_ago = casos_raw.iloc[-1,-8]
total_deaths_one_week_ago = deaths_one_week_ago.loc["Total", "Fallecidos"]

In [13]:
#hide
diff_cases = total_confirmed-total_confirmed_one_week_ago
diff_deaths = total_deaths-total_deaths_one_week_ago

In [14]:
#hide
total_confirmed = '{:,}'.format(total_confirmed) #.replace(',', '.')
total_deaths = '{:,}'.format(total_deaths) #.replace(',', '.')

In [15]:
#hide
diff_cases = '{:,}'.format(diff_cases) #.replace(',', '.')
diff_deaths = '{:,}'.format(diff_deaths) #.replace(',', '.')

In [16]:
#hide_input
HTML(f'<p style="color:#3361ff;"><span style="font-weight:bold;">Confirmed cases:</span> {total_confirmed} (+{diff_cases} since 7 days ago)</p><p style="color:#FF3F3F;"><span style="font-weight:bold;">Confirmed deaths:</span> {total_deaths} (+{diff_deaths} since 7 days ago)</p>')

In [17]:
#hide_input
(data_table.style.set_properties(**{'text-align': 'right'}).background_gradient(cmap='Reds').hide_index()).set_caption(
    'Statistics by region: Confirmed cases and confirmed deaths')

Region,Confirmed cases,"Confirmed cases per 100,000 people",Confirmed deaths,Confirmed deaths per 100.000 people
Arica y Parinacota,265,66.100000,3,1.200000
Tarapacá,164,121.900000,1,0.300000
Antofagasta,457,105.100000,4,0.600000
Atacama,35,11.100000,0,0.000000
Coquimbo,74,6.500000,0,0.000000
Valparaíso,485,42.400000,9,0.500000
Metropolitana,7858,8.900000,95,1.200000
O’Higgins,94,53.500000,1,0.100000
Maule,363,44.400000,12,1.100000
Ñuble,741,376.200000,14,2.700000


# Evolution of total confirmed cases by region

In [18]:
#hide
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [19]:
#hide
data_raw = data_raw.drop("Total")

In [20]:
#hide
data = pd.DataFrame()
i = 0
for date in data_raw.keys():
    for n, region in enumerate(data_raw.index):
        data[i] = date, region, data_raw[date].loc[region], n
        i += 1

In [21]:
#hide
data_raw = data_raw.reset_index()
regiones = data_raw['Region'].values
data = data.T
data = data.rename(columns={0: "date", 1: "region", 2: "casos", 3: "codigo region"})

In [22]:
#hide
data["casos"] = data["casos"].astype(int)

In [23]:
#hide_input
input_dropdown = alt.binding_select(options=data['region'].unique())
selection1 = alt.selection_single(fields=['region'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['region'], on='mouseover')
color = alt.condition(selection1 | selection2,
                    alt.Color('region:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

chart = alt.Chart(data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Date')),
    y=alt.Y('casos', axis=alt.Axis(title='Confirmed cases')),
    color=color,
    tooltip=['date', 'region', 'casos'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
).properties(
    title='COVID-19 in Chile: Total confirmed cases by región'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(data).mark_point().encode(
    y=alt.Y('region:N', axis=alt.Axis(orient='right'), sort=regiones),
    color=color
).add_selection(
    selection1, selection2
)

chart.properties(width=600, height=400) | legend

alt.HConcatChart(...)

# Evolution of total confirmed deaths by region

In [24]:
#hide
data = pd.DataFrame()
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-03-24-CasosConfirmados-totalRegional.csv",
    index_col='Region')
data['2020-03-24'] = data_raw['Fallecidos']

In [25]:
#hide
first_death_date = '2020-03-24'
total_days = (pd.to_datetime(today)-pd.to_datetime(first_death_date)).days

In [26]:
#hide
for i in np.arange(total_days+1):
  date = (pd.to_datetime(first_death_date)+pd.DateOffset(i)).strftime('%Y-%m-%d')
  s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" + date + "-CasosConfirmados-totalRegional.csv"
  data_by_date = pd.read_csv(s)
  if 'Fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Fallecidos"].values
  elif 'Casos fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Casos fallecidos"].values
  else:
    data[date] = data_by_date[" Casos fallecidos"].values

In [27]:
#hide
data = data.drop("Total")

In [28]:
#hide
new_data = pd.DataFrame()
i = 0
for date in data.keys():
    for n, region in enumerate(data.index):
        new_data[i] = date, region, data[date].loc[region], n
        i += 1

In [29]:
#hide
data = data.reset_index()
regiones = data['Region'].values
new_data = new_data.T
new_data = new_data.rename(columns={0: "date", 1: "region", 2: "fallecidos", 3: "codigo region"})

In [30]:
#hide
new_data["fallecidos"] = new_data["fallecidos"].astype(int)

In [31]:
#hide_input

input_dropdown = alt.binding_select(options=new_data['region'].unique())
selection1 = alt.selection_single(fields=['region'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['region'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('region:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(new_data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Date')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Confirmed deaths')),
    color=color,
    tooltip=['date', 'region', 'fallecidos'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
    ).properties(
    title='COVID-19 in Chile: Total confirmed deaths by region'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(new_data).mark_point().encode(
    y=alt.Y('region:N', axis=alt.Axis(orient='right'), sort=regiones),
    color=color
).add_selection(
    selection1, selection2
)

bars.properties(width=600, height=400) | legend

alt.HConcatChart(...)

In [32]:
#hide
# Window size cases
WS_cases = 7

In [33]:
#hide
def my_cases_plot(region):
    aux1 = casos_raw.loc[region]
    aux2 = [t - s for s, t in zip(aux1, aux1.loc['2020-03-04':])]
    # Create the dataframe
    new_cases = pd.DataFrame()
    new_cases["Date"] = casos_raw.loc[region].reset_index()["index"].iloc[1:].values
    new_cases["Confirmed cases"] = aux2
    new_cases["Rolling"] = new_cases["Confirmed cases"].rolling(window=WS_cases).mean()
    new_cases["Daily confirmed cases"] = len(new_cases) * ["Daily confirmed cases"]
    new_cases["7-day rolling average"] = len(new_cases) * ["7-day rolling average"]
    # Make the plot
    bars = alt.Chart(new_cases).mark_bar(opacity=0.7).encode(
        x = alt.X('Date:N', axis=alt.Axis(title='Date')),
        y = alt.Y('Confirmed cases:Q', axis=alt.Axis(title='Daily confirmed cases')),
        tooltip = ['Date', 'Confirmed cases'],
        opacity=alt.Opacity('Daily confirmed cases', legend=alt.Legend(title=""))
    )

    line = alt.Chart(new_cases).mark_line(point={
        "filled": True,
        "fill": "#1f77b4"
    }, color="#1f77b4").encode(
        x=alt.X('Date:N', axis=alt.Axis(title='Date')),
        y = alt.Y('Rolling:Q'),
        shape=alt.Shape('7-day rolling average', legend=alt.Legend(title=""))
    )
    return bars, line

Data source: [Science Ministry](https://github.com/MinCiencia/Datos-COVID19).

# Daily confirmed cases

## Chile

In [34]:
#hide
region = "Total"

In [35]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in Chile',
    width=600
)

alt.LayerChart(...)

## Arica y Parinacota

In [36]:
#hide
region = "Arica y Parinacota"

In [37]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Tarapacá

In [38]:
#hide
region = "Tarapacá"

In [39]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Antofagasta

In [40]:
#hide
region = "Antofagasta"

In [41]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Atacama

In [42]:
#hide
region = "Atacama"

In [43]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Coquimbo

In [44]:
#hide
region = "Coquimbo"

In [45]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Valparaíso

In [46]:
#hide
region = "Valparaíso"

In [47]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Metropolitana

In [48]:
#hide
region = "Metropolitana"

In [49]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## O'Higgins

In [50]:
#hide
region = "O’Higgins"

In [51]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Maule

In [52]:
#hide
region = "Maule"

In [53]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Ñuble

In [54]:
#hide
region = "Ñuble"

In [55]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Biobío

In [56]:
#hide
region = "Biobío"

In [57]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Araucanía

In [58]:
#hide
region = "Araucanía"

In [59]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Los Ríos

In [60]:
#hide
region = "Los Ríos"

In [61]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Los Lagos

In [62]:
#hide
region = "Los Lagos"

In [63]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Aysén

In [64]:
#hide
region = "Aysén"

In [65]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

## Magallanes

In [66]:
#hide
region = "Magallanes"

In [67]:
#hide_input
bars, line = my_cases_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed cases in {region}',
    width=600
)

alt.LayerChart(...)

# Daily confirmed deaths

In [68]:
#hide
data = pd.DataFrame()
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-03-24-CasosConfirmados-totalRegional.csv",
    index_col='Region')
data['2020-03-24'] = data_raw['Fallecidos']

In [69]:
#hide
first_death_date = '2020-03-24'
total_days = (pd.to_datetime(today)-pd.to_datetime(first_death_date)).days

In [70]:
#hide
for i in np.arange(total_days+1):
  date = (pd.to_datetime(first_death_date)+pd.DateOffset(i)).strftime('%Y-%m-%d')
  s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" + date + "-CasosConfirmados-totalRegional.csv"
  data_by_date = pd.read_csv(s)
  if 'Fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Fallecidos"].values
  elif 'Casos fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Casos fallecidos"].values
  else:
    data[date] = data_by_date[" Casos fallecidos"].values

In [71]:
#hide
WS_deaths = 7

In [72]:
#hide
def my_deaths_plot(region):
    aux1 = data.loc[region]
    aux2 = [t - s for s, t in zip(aux1, aux1.loc['2020-03-25':])]
    new_deaths = pd.DataFrame()
    new_deaths["Dates"] = data.loc["Total"].reset_index()["index"].iloc[1:].values
    new_deaths["New_deaths"] = aux2
    new_deaths["Rolling"] = new_deaths["New_deaths"].rolling(window=WS_deaths).mean()
    new_deaths["Daily confirmed deaths"] = len(new_deaths) * ["Daily confirmed deaths"]
    new_deaths["7-day rolling average"] = len(new_deaths) * ["7-day rolling average"]
    bars = alt.Chart(new_deaths).mark_bar(opacity=0.7, color='firebrick').encode(
        x = alt.X('Dates:N', axis=alt.Axis(title='Date')),
        y = alt.Y('New_deaths:Q', axis=alt.Axis(title='Daily confirmed deaths')),
        tooltip = ['Dates', 'New_deaths'],
        opacity=alt.Opacity('Daily confirmed deaths', legend=alt.Legend(title=""))
    )

    line = alt.Chart(new_deaths).mark_line(point={
          "filled": True,
          "fill": "firebrick"
        }, color='firebrick').encode(
        x=alt.X('Dates:N', axis=alt.Axis(title='Date')),
        y = alt.Y('Rolling:Q'),
        shape=alt.Shape('7-day rolling average', legend=alt.Legend(title=""))
    )
    return bars, line

## Chile

In [73]:
#hide
region = "Total"

In [74]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in Chile',
    width=600
)

alt.LayerChart(...)

## Arica y Parinacota

In [75]:
#hide
region = "Arica y Parinacota"

In [76]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Tarapacá

In [77]:
#hide
region = "Tarapacá"

In [78]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Antofagasta

In [79]:
#hide
region = "Antofagasta"

In [80]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Atacama

In [81]:
#hide
region = "Atacama"

In [82]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Coquimbo

In [83]:
#hide
region = "Coquimbo"

In [84]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Valparaíso

In [85]:
#hide
region = "Valparaíso"

In [86]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Metropolitana

In [87]:
#hide
region = "Metropolitana"

In [88]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## O’Higgins

In [89]:
#hide
region = "O’Higgins"

In [90]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Maule

In [91]:
#hide
region = "Maule"

In [92]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Ñuble

In [93]:
#hide
region = "Ñuble"

In [94]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Biobío

In [95]:
#hide
region = "Biobío"

In [96]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Araucanía

In [97]:
#hide
region = "Araucanía"

In [98]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Los Ríos

In [99]:
#hide
region = "Los Ríos"

In [100]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Los Lagos

In [101]:
#hide
region = "Los Lagos"

In [102]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Aysén

In [103]:
#hide
region = "Aysén"

In [104]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)

## Magallanes

In [105]:
#hide
region = "Magallanes"

In [106]:
#hide_input
bars, line = my_deaths_plot(region)
(bars + line).properties(
    title=f'COVID-19 in Chile: Daily confirmed deaths in {region}',
    width=600
)

alt.LayerChart(...)